# Exercice: Modélisation orientée requête (Cassandra)

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/cassandra.png" width="100" height="100"></td>
    </tr></table>



### Dans ce notebook, nous allons travailler sur la modélisation orientée requête sous la BDD No-SQL Cassandra, en particulier nous traitons les points suivants :

* Dénormaliser une BDD relationnelle en partant du principe 1 table per query 
* Définir la partie Primary Key pour chaque requête
* Optimiser vos modèles en choisissant la bonne Partition Key & Clustering Column
* Valider vos modèles 
* Tester quelques requêtes particulières


#### installer cassandra-driver afin de se connecter au cluster cassandra
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [46]:
import cassandra
import random

### créer une connection au cluster cassandra

In [6]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) # si cassandra est installé localement avec le port par défaut 9042
    session = cluster.connect()
except Exception as e:
    print(e)

### créer maintenant un keyspace nommé "iot_esi"

In [7]:
try:
    
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS iot 
    WITH REPLICATION = {'class':'SimpleStrategy','replication_factor':1}
    """);
 

except Exception as e:
    print(e)

### connecter à votre Keyspace  iot_esi 


In [8]:
try:
    
    session.set_keyspace("iot")
   
    
except Exception as e:
    print(e)

### Supposons qu'on a un système IoT qui stocke les données capturées par les différents Sensors en utilisant un modèle RELATIONNEL, tel que est décrit dans le schéma ER ci-dessous:

<table><tr>
    <td><img src="image/ex1_IoT_ER_model.png" width="500" height="500"></td>
      </tr></table>

### imaginons  que nous souhaitons analyser ces données à  travers 3 requêtes analytiques:
* `Query1= retourner les informations(name, nbSensors) de tous les networks d'une région donnée`
* `Query2= retourner les informations (name,type,localisation, characteristics) de tous les Sensors d'un Network donné.`
* `Query3= retourner le AVG_measurements par sensor,  par année et par mois.  Ordonner le résultat par  année(desc) et par mois(asc)`

#### Créer les trois tables qui correspondent aux requêtes précédentes, en indiquant pour chacune:
* Primary Key
* Partition Key
* Clustering Column

In [21]:
try:
    
    # retourner les informations(name, nbSensors) de tous les networks d'une région donnée
    
    session.execute("drop table if exists network_by_region")
   
    session.execute("""
        CREATE TABLE Network_By_Region (
            region text,
            idNetwork int,
            name text,
            nbSensors int,
            PRIMARY KEY (region, idNetwork)
        );

        """)
     #the query1 will be in cql:
    # SELECT name , nbSensors FROM Network_By_Region WHERE region = 'West'; 
    

   
    
except Exception as e:
        print(e)

In [36]:
try:
    
    # retourner les informations (name,type,localisation, characteristics) de tous les Sensors d'un Network donné.`
    session.execute("DROP TABLE IF EXISTS Sensor_By_Network")
    session.execute("""
        CREATE TABLE Sensor_By_Network (
            idNetwork int,
            
            idSensor int,
            name text,
            type text,
            latitude double,
            longitude double,
            characteristics map<text,text>,
            PRIMARY KEY (idNetwork, idSensor)
        );
    """)
    
    # query2 in cql is:
    #SELECT name, type, latitude, longitude, characteristics FROM Sensor_By_Network WHERE idNetwork = '<idNetwork>';
    
   
    
    
except Exception as e:
        print(e)

In [44]:
try:
    
    
    ### retourner le AVG_measurements par sensor,  par année et par mois.  Ordonner le résultat par  année(desc) et par mois(asc)
     
    session.execute("drop table if exists RecordedData_By_Year_Month")
    session.execute("""
        CREATE TABLE RecordedData_By_Year_Month (
            idSensor int,
            year int,
            month int,
            value double,
            PRIMARY KEY (idSensor, year, month)
        ) 
        WITH CLUSTERING ORDER BY (year DESC, month ASC);
    """)
    # note that we stored the value and not the calculated average cuz if we do so 
    # we will be obliged to calculate the averages every time a new record of data is added which is coasty
    # instead we can calculate the average only when we need it (for analytical purposes which happens less often)
    # cql query is:
    #SELECT idSensor, year, month, AVG(value) FROM RecordedData_By_Year_Month GROUP BY idSensor, year, month ORDER BY year DESC, month ASC;
except Exception as e:
        print(e)

### Une fois vous créez les différentes tables, essayez de les peupler en se basant sur le contenu de la BDD relationnelle.

<table><tr>
    <td><img src="image/data_tables.png" width="600" height="500"></td></tr>
  
</table>

### Insertion des données dans Table1

In [38]:
query = "INSERT INTO Network_By_Region (region, idNetwork, name, nbSensors) "
query = query + " VALUES (%s,%s,%s,%s)"

try:
    session.execute(query, ('ouest',1,'esi-net',10))
    session.execute(query, ('ouest',2,'udl-net',5))
    session.execute(query, ('centre',1,'alger-net',15))
    
except Exception as e:
        print(e)

### Insertion des données dans Table2

In [43]:
query = "INSERT INTO Sensor_By_Network (idNetwork, idSensor, name, type, latitude, longitude, characteristics) "
query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"

try:
    session.execute(query, (1,1,'temp-11','temp',31,-96,{"accuracy":"medium","sensitivity":"high"}))
    session.execute(query, (1,2,'temp-12','temp',31,-97,{"accuracy":"high","sensitivity":"high"}))
    session.execute(query, (2,3,'temp-21','temp',32,-97,{"accuracy":"low","sensitivity":"medium"}))
    session.execute(query, (3,4,'temp-31','temp',44,-111,{"accuracy":"medium","sensitivity":"medium"}))
    session.execute(query, (3,5,'temp-32','temp',45,-111,{"accuracy":"high","sensitivity":"low"}))
    
except Exception as e :
        print(e) 


### Insertion des données dans Table3

In [128]:
query = "INSERT INTO RecordedData_By_Year_Month_query4 (idSensor, year, month, value) "
query = query + " VALUES (%s,%s,%s,%s)"

try:
    for i in range(1, 50):
        idSensor = random.randint(1, 10)
        year = random.randint(2020, 2023)
        month = random.randint(1, 12)
        value = random.uniform(0, 100)
        session.execute(query, (idSensor, year, month, value))

except Exception as e:
        print(e)

### Valider votre modèle en exécutant les requêtes suivantes

* Retourner le nombre de sensors dans la région ouest"

* Retourner les sensors qui sont placés dans le network "esi-net"

* Retourner la température maximale enregistrée par le sensor "temp-11" pendant l'année 2021

In [107]:
# query1: Retourner le nombre de sensors dans la région ouest
query = "SELECT SUM(NbSensors)  from Network_By_Region where region = 'ouest'"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)           

Row(system_sum_nbsensors=15)


In [106]:
# query2: Retourner les sensors qui sont placés dans le network "esi-net"
query = "select * from Sensor_By_Network where idNetwork = 1"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)    

Row(idnetwork=1, idsensor=1, characteristics=OrderedMapSerializedKey([('accuracy', 'medium'), ('sensitivity', 'high')]), latitude=31.0, longitude=-96.0, name='temp-11', type='temp')
Row(idnetwork=1, idsensor=2, characteristics=OrderedMapSerializedKey([('accuracy', 'high'), ('sensitivity', 'high')]), latitude=31.0, longitude=-97.0, name='temp-12', type='temp')


In [105]:
# query3: Retourner la température maximale enregistrée par le sensor "temp-11" pendant l'année 2021
# (for test) query = "select *  from RecordedData_By_Year_Month where idSensor = 1 order by  year desc, month"
query = "select max(value)  from RecordedData_By_Year_Month where idSensor = 1 and year = 2021"
try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)         

Row(system_max_value=84.77362110188696)


### Essayons d'autres requêtes 

* `Retourner la température maximale enregistrée par le sensor "temp-11"  pendant le  mois d'octobre de toutes les années` 

In [100]:
# query4
query = "select max(value)  from RecordedData_By_Year_Month where idSensor = 1 and month = 10 "



try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)      

Error from server: code=2200 [Invalid query] message="PRIMARY KEY column "month" cannot be restricted as preceding column "year" is not restricted"


### Proposer une solution pour optimiser cette requête

In [121]:
try:
    
    
    ### retourner le AVG_measurements par sensor,  par année et par mois.  Ordonner le résultat par  année(desc) et par mois(asc)
     
    session.execute("drop table if exists RecordedData_By_Year_Month_query4")
    session.execute("""
        CREATE TABLE RecordedData_By_Year_Month_query4 (
            idSensor int,
            year int,
            month int,
            value double,
            PRIMARY KEY (idSensor, month )
        ) 
        WITH CLUSTERING ORDER BY (  month ASC);
    """)
    # note that we stored the value and not the calculated average cuz if we do so 
    # we will be obliged to calculate the averages every time a new record of data is added which is coasty
    # instead we can calculate the average only when we need it (for analytical purposes which happens less often)
    # cql query is:
    #SELECT idSensor, year, month, AVG(value) FROM RecordedData_By_Year_Month GROUP BY idSensor, year, month ORDER BY year DESC, month ASC;
except Exception as e:
        print(e)

In [122]:
# query4
query = "select max(value)  from RecordedData_By_Year_Month_query4 where idSensor = 1 and month = 10 "



try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)  

Row(system_max_value=None)


###  drop  tables. 

In [40]:
query = "drop table IF EXISTS ..."
rows = session.execute(query)

query = "drop table IF EXISTS ... "
rows = session.execute(query)

query = "drop table IF EXISTS ....  "
rows = session.execute(query)



query = "drop table IF EXISTS ... "
rows = session.execute(query)

### close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()